# Data Cleanup: GEOID tables

The goal of this notebook is to produce one clean DataFrame of GEOID codes for US states and counties and separate one for "places". 

##  GEOIDs and why we love them
*Geographic Identifier (GEOID)* is a general term for the unique numeric codes used to identify geographic entities. Specifically, the Census Bureau uses FIPS (Federal Information Processing Series) codes to identify most of the geographies for which it tabulates data. Because these codes are maintained by ANSI (the American National Standards Institute) we might hear them referred to as "ANSI", "FIPS", or even "ANSI/FIPS" codes. For clarity, we'll refer to them as "FIPS" codes. Census tract codes are the exception - these are maintained by the Census Bureau itself, so we'll refer to them as "census tract codes." Regardless of what we call them, GEOIDs are the key piece of information that enables us to tie our population data to our geographic data. 

A unique GEOID can be derived for every piece of geography in the US by concatenating its code with those of the geographies in which it nests. The geography we'll be mapping, the *census tract*, nests within a *county*, which nests within a *state*. So, to derive a census tract's unique GEOID we concatenate its: STATE FIPS code + COUNTY FIPS code + CENSUS TRACT code.

For example, the GEOID for the census tract in Brooklyn where I sit writing this is:

```
STATE + COUNTY + TRACT
NY + Kings County + Tract #
36 + 047 + 016500 
= 36047016500

```

## Which GEOIDs do we need?

Given that the dataset required to map every census tract in the US is *enormous*, we'll narrow our initial mapping exercise to a subsection of particular interest within the indoor farming industry - New York City. We will, however, write our code, so that additional geographies can easily be added later. To begin, we'll need list of state, county, and "place" FIPS codes. (Interestingly, the hierarchy of census geographies doesn't actually contain a distinct "city" category, rather it categorizes New York City as a "place", thus our need for the place FIPS codes list.)

#### *Data sources*

Files downloaded from Census Bureau FTP server via FTP client.

*State FIPS Codes* 
* file location: ftp://ftp2.census.gov/geo/docs/reference/codes/state.txt

*County FIPS Codes*
* file location: ftp://ftp2.census.gov/geo/docs/reference/codes/national_county.txt

*Place FIPS Codes*
* file location: ftp://ftp2.census.gov/geo/docs/reference/codes/PLACElist.txt

#### *Some helper code for managing file paths*
We'll use the code below throughout this project to make it easier to refer to the folders where our various data files are stored.

In [1]:
# os and patlib modules used to make it easier to refer to project folders 

import os, pathlib
base_dir = pathlib.Path(os.getcwd()).parent
data_archive_dir = os.path.join(base_dir, "data_archive")
clean_data_dir = os.path.join(data_archive_dir, "clean")
data_dir = os.path.join(base_dir, "data")
shapes_dir = os.path.join(data_dir,"shapes")
json_dir = os.path.join(data_dir,"geojson")
util_dir = os.path.join(data_dir,"util")

## State FIPS codes

Let's go ahead and load as a DataFrame the list of codes for the states and state equivalents and examine the first few rows. We'll include just the columns we want, rename them to better describe their contents, and reorder them as we like. 

In [2]:
import pandas as pd

# these options determine how much data is displayed in the notebook
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

file_state = os.path.join(data_archive_dir, 'state.txt')

df_state = pd.read_csv(file_state, 
                       usecols=['STATE', 'STUSAB', 'STATE_NAME'], # use only these columns
                       delimiter="|", # load txt file as pandas DataFrame 
                       encoding="iso-8859-1", 
                       encoding_errors='ignore')[['STATE_NAME', 'STUSAB', 'STATE',]] # reorder cols

df_state.rename(columns={'STATE': 'State FIPS', 'STUSAB': 'State', 'STATE_NAME': 'State Name'}, inplace=True) # rename columns
df_state.head(3)

,State Name,State,State FIPS
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4


Now let's proceed with our usual, thorough search for missing values.

In [3]:
df_state.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   State Name  57 non-null     object
 1   State       57 non-null     object
 2   State FIPS  57 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.5+ KB


<!-- ****DOUBLE CHECK - do we want State FIPS as integer or string? Population data has GEOIDs as strings. What about shapefiles? Let's leave it for now.****

Hmm. "State" and "State Name" are of type "object", which makes sense since our visual inspection makes it clear they are strings. State FIPS, however, is showing up as an integer type. This makes sense, but since our population data has GEOIDs formatted as strings, we'll follow its lead and change the dtype here, so we'll be able match on this column if needed in the future.  -->

According to `.info()` our list has no null values, let's double check this using `.isna().sum().`

In [4]:
df_state.isna().sum()

State Name    0
State         0
State FIPS    0
dtype: int64

Lastly, let's confirm that our datatypes make sense. 

In [5]:
df_state.dtypes

State Name    object
State         object
State FIPS     int64
dtype: object

Our dtypes looks good. "State Name" and "State" column values are strings, so dtype "object" is correct here. State FIPS codes are integers, so the dtype "int64" is correct here. Now, we're ready to move onto county FIPS codes.  

## County FIPS codes

Let's load as a DataFrame our list of County FIPS codes. Again, we'll limit it to the columns we need - reorder and rename them, then perform a quick visual inspection. 

In [6]:
# this text file uses UTF-8 encoding 
file_national_county = os.path.join(data_archive_dir, 'national_county.txt')

df_county = pd.read_csv(file_national_county,
                        delimiter=",",
                        usecols=['State ANSI', 'County ANSI', 'County Name'], # use only these columns
                        encoding="utf-8", # QUESTION: Patrick, this text file uses UTF-8 encoding, what should we use here?
                        encoding_errors='ignore')[['State ANSI', 'County Name', 'County ANSI']] # reorder columns

# rename columns
df_county.columns = ['State FIPS', 'County', 'County FIPS']
df_county.head(3)

,State FIPS,County,County FIPS
0,1,Autauga County,1
1,1,Baldwin County,3
2,1,Barbour County,5


In [7]:
df_county.shape

(3235, 3)

Now, let's proceed with our thorough search for missing values.

In [8]:
df_county.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3235 entries, 0 to 3234
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   State FIPS   3235 non-null   int64 
 1   County       3235 non-null   object
 2   County FIPS  3235 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 75.9+ KB


In [9]:
df_county.isna()

,State FIPS,County,County FIPS
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
3230,False,False,False
3231,False,False,False
3232,False,False,False
3233,False,False,False


In [10]:
df_county.isna().sum()

State FIPS     0
County         0
County FIPS    0
dtype: int64

And now a quick check to confirm that our datatypes make sense. 

In [11]:
df_county.dtypes

State FIPS      int64
County         object
County FIPS     int64
dtype: object

## State & county FIPS codes combined

Now, let's consolidate our state and county FIPS codes DataFrames into a new DataFrame and call it, *df_state_county*. We want to include the "State Names" column from our *df_states*, as the 2-letter abbreviations for some island territories on our list may be less familiar. 

In [12]:
df_state_county = pd.merge(df_state, df_county, on=['State FIPS'], how='left') 
df_state_county.head(3)

,State Name,State,State FIPS,County,County FIPS
0,Alabama,AL,1,Autauga County,1
1,Alabama,AL,1,Baldwin County,3
2,Alabama,AL,1,Barbour County,5


## Addressing non-ASCII characters

While we're pretty confident that our state names and abbreviations contain only standard ASCII characters, we suspect that some county names may include characters with diacritic marks. We need to ensure consistent treatment of these characters between files, especially if we wish to perform merges on these columns at any point. So, let's check *df_state_county* now for any characters with diacritic marks, so we can decide if and how we'd like to handle them.

In [13]:
# this function checks for characters with diacritic marks

def is_non_ascii(value):
    if type(value) != str:
        return True
    try:
        value.encode('ascii')
        return False
    except:
        return True

In [14]:
# call our function 'is_non_ascii' to check for characters 
# with diacritics    

df_state_county.loc[ (df_state_county["State Name"].apply(is_non_ascii)) | (df_state_county["County"].apply(is_non_ascii)) ]

,State Name,State,State FIPS,County,County FIPS


All clear! No diacritics found in *df_state_county*, just standard ASCII characters. Let's move on to our list of Place FIPS codes. 

## Place FIPS codes

As usual, we'll load our list as a DataFrame, rename and reorder columns, then inspect a few rows. 

In [15]:
# this text file uses ANSI encoding
file_place = os.path.join(data_archive_dir, 'PLACElist.txt')

df_place = pd.read_csv(file_place, 
                       delimiter="|", 
                       usecols=['STATE', 'STATEFP', 'PLACEFP', 'PLACENAME', 'COUNTY'], # use only these columns
                       encoding="iso-8859-1" # QUESTION: Patrick, this txt file uses ANSI encoding, what should we use here?
                       )[['STATE','STATEFP', 'PLACENAME', 'PLACEFP', 'COUNTY']] # reorder columns
#encoding_errors='ignore'
df_place.rename(columns={'STATE': 'State', 'STATEFP': 'State FIPS', 'PLACENAME': 'Place', 'PLACEFP': 'Place FIPS', 'COUNTY': 'County'}, inplace=True) # rename columns
                         
df_place.head(3) 

,State,State FIPS,Place,Place FIPS,County
0,AL,1,Abanda CDP,100,Chambers County
1,AL,1,Abbeville city,124,Henry County
2,AL,1,Adamsville city,460,Jefferson County


In [16]:
df_place.dtypes

State         object
State FIPS     int64
Place         object
Place FIPS     int64
County        object
dtype: object

While all of our state names contain only standard ASCII characters, depending on how our file is formatted, it's possible that some of our place or county names may include characters with diacritic marks. Let's take a closer look now using our `is_non_ascii` function.

In [17]:
# check for characters with diacritic marks

def is_non_ascii(value):
    if type(value) != str:
        return True
    try:
        value.encode('ascii')
        return False
    except:
        return True
    
df_place.loc[ (df_place["County"].apply(is_non_ascii)) | (df_place["Place"].apply(is_non_ascii)) ]

,State,State FIPS,Place,Place FIPS,County
2599,CA,6,La Cañada Flintridge city,39003,Los Angeles County
2982,CA,6,Piñon Hills CDP,57302,San Bernardino County
3506,CO,8,Cañon City city,11810,Fremont County
22545,NM,35,Anthony CDP,3820,Doña Ana County
22561,NM,35,Berino CDP,6830,Doña Ana County
...,...,...,...,...,...
41395,PR,72,Tallaboa comunidad,81413,Peñuelas Municipio
41396,PR,72,Tallaboa Alta comunidad,81456,Peñuelas Municipio
41399,PR,72,Tierras Nuevas Poniente comunidad,82187,Manatí Municipio
41405,PR,72,Vázquez comunidad,85111,Salinas Municipio


Well, it appears there are more than a few diacritic marks in *df_place*! Let's address them by mapping each to its closest approximate in the English 26-letter alphabet. This will ensure consistency with our *df_state_county* DataFrame.

In [18]:
import json

# open a json file of characters with diacritics mapped to their
# closest approximate in the English alphabet
# create a dictionary of these character mappings
diacritic_mapping_file = os.path.join(data_dir,"util/diacritic_translate.json")
with open(diacritic_mapping_file, "r") as mapping_file:
    diac_char_map = json.load(mapping_file)

# now we need the integer ordinal of each character to use with 
# pandas Series str.translate    
# create an empty dictionary to store these
diac_ord_map = dict()

# iterate over the entries in the character mappings dictionary
for k,v in diac_char_map.items():
    # populate the ordinal dict with the ordinals
    # of each key => value character
    diac_ord_map[ord(k)] = ord(v)

In [19]:
# use pandas Series.str.translate to translate all our County and Place names
# with diacritics
df_place['County'] = df_place['County'].str.translate(diac_ord_map)
df_place['Place'] = df_place['Place'].str.translate(diac_ord_map)

We can confirm that our character replacement worked by spot checking some rows we know contained characters with diactric marks. 

In [20]:
# this row previously had a diacritic mark in the "Place" column
df_place.loc[df_place['Place FIPS'] == 39003]

,State,State FIPS,Place,Place FIPS,County
2599,CA,6,La Canada Flintridge city,39003,Los Angeles County


In [21]:
# this row previously contained a diacritic in the "County" column
df_place.loc[df_place['Place FIPS'] == 81413]

,State,State FIPS,Place,Place FIPS,County
41395,PR,72,Tallaboa comunidad,81413,Penuelas Municipio


Our diacritic marks have been taken care of. Now, let's add to *df_place* the "State Name" column, by merging in *df_state*.

In [22]:
# df_place = pd.merge(df_place, df_state_county ) 
df_place = pd.merge(df_place, df_state) 
df_place = df_place[['State Name', 'State', 'State FIPS', 'Place', 'Place FIPS', 'County',]] # reorder columns
df_place.head(3)

,State Name,State,State FIPS,Place,Place FIPS,County
0,Alabama,AL,1,Abanda CDP,100,Chambers County
1,Alabama,AL,1,Abbeville city,124,Henry County
2,Alabama,AL,1,Adamsville city,460,Jefferson County


We can now perform our search for missing values and incorrect data types.

In [23]:
df_place.info() # check for missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41414 entries, 0 to 41413
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   State Name  41414 non-null  object
 1   State       41414 non-null  object
 2   State FIPS  41414 non-null  int64 
 3   Place       41414 non-null  object
 4   Place FIPS  41414 non-null  int64 
 5   County      41414 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.2+ MB


In [24]:
df_place.isna().sum() # another check for missing values

State Name    0
State         0
State FIPS    0
Place         0
Place FIPS    0
County        0
dtype: int64

In [25]:
df_place.dtypes # confirming data types are correct

State Name    object
State         object
State FIPS     int64
Place         object
Place FIPS     int64
County        object
dtype: object

Looks good! No missing value and our data types make sense. Since we'll be mapping the place "New York City" in our initial mapping exercise, let's take a look at places that begin with the words "New York".

In [26]:
df_place.loc[df_place['Place'].str.startswith("New York")]

,State Name,State,State FIPS,Place,Place FIPS,County
17317,Minnesota,MN,27,New York Mills city,46060,Otter Tail County
17318,Minnesota,MN,27,New York Mills city,46060,Otter Tail County
24316,New York,NY,36,New York city,51000,"Bronx County, Kings County, New York County, Queens County, Richmond County"
24317,New York,NY,36,New York Mills village,51011,Oneida County


Strangely, the word "city" in the name "New York city" is not capitalized. Let's change this throughout our DataFrame, so that every place name is in titlecase. 

In [27]:
df_place['Place'] = df_place['Place'].str.title()
df_place.loc[df_place['Place'].str.startswith("New York")]

,State Name,State,State FIPS,Place,Place FIPS,County
17317,Minnesota,MN,27,New York Mills City,46060,Otter Tail County
17318,Minnesota,MN,27,New York Mills City,46060,Otter Tail County
24316,New York,NY,36,New York City,51000,"Bronx County, Kings County, New York County, Queens County, Richmond County"
24317,New York,NY,36,New York Mills Village,51011,Oneida County


Done. We now have one clean DataFrame of state and county FIPS codes and another with place FIPS codes. We can go ahead have save them both as Parquet files. 

In [28]:
# save DF as Parquet 
df_state_county.head(3)
clean_codes_state_county_file = os.path.join(clean_data_dir,'state_county.parquet')
df_state_county.to_parquet(clean_codes_state_county_file, compression='BROTLI')

In [29]:
# save DF as Parquet 
df_place.head(3)
clean_codes_place_file = os.path.join(clean_data_dir,'place.parquet')
df_place.to_parquet(clean_codes_place_file, compression='BROTLI')